In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.datasets import mnist
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
import random


2024-02-04 16:08:29.803330: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
def create_network(network_params):
    model = Sequential()
    model.add(Dense(network_params['first_layer_neurons'], activation=network_params['activation'], input_shape=(784,)))
    for i in range(network_params['hidden_layers']):
        model.add(Dense(network_params['hidden_layer_neurons'], activation=network_params['activation']))
    model.add(Dense(10, activation='softmax'))  # 10 sınıf için çıkış katmanı
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model

def generate_initial_population(pop_size):
    population = []
    for _ in range(pop_size):
        network_params = {
            'first_layer_neurons': random.choice([32, 64, 128]),
            'hidden_layers': random.choice([1, 2, 3]),
            'hidden_layer_neurons': random.choice([32, 64, 128]),
            'activation': random.choice(['relu', 'tanh'])
        }
        population.append(network_params)
    return population

def compute_fitness(model, X_train, y_train, X_val, y_val):
    model.fit(X_train, y_train, epochs=3, verbose=0)  # Eğitim süresini kısa tutmak için 3 epoch
    loss, accuracy = model.evaluate(X_val, y_val, verbose=0)
    return accuracy

def select_parents(population, fitness, num_parents):
    parents = np.argsort(fitness)[-num_parents:]
    return [population[i] for i in parents]

def crossover(parents):
    offspring = []
    for _ in range(len(parents) // 2):
        parent1, parent2 = random.sample(parents, 2)
        child = parent1.copy()
        for key in parent2:
            if random.uniform(0, 1) < 0.5:
                child[key] = parent2[key]
        offspring.append(child)
    return offspring

def mutate(offspring):
    for child in offspring:
        if random.uniform(0, 1) < 0.1:  # %10 mutasyon şansı
            child['first_layer_neurons'] = random.choice([32, 64, 128])
            child['hidden_layers'] = random.choice([1, 2, 3])
            child['hidden_layer_neurons'] = random.choice([32, 64, 128])
            child['activation'] = random.choice(['relu', 'tanh'])
    return offspring


In [3]:
# MNIST veri setini yükleme
(X_train, y_train), (X_test, y_test) = mnist.load_data()
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.1)
X_train = X_train.reshape(X_train.shape[0], 784)
X_val = X_val.reshape(X_val.shape[0], 784)
y_train = to_categorical(y_train, 10)
y_val = to_categorical(y_val, 10)


In [4]:

# Genetik algoritma parametreleri
population_size = 10
num_generations = 5
num_parents = 4

# Başlangıç popülasyonunu oluşturma
population = generate_initial_population(population_size)




In [5]:
# Her nesil için döngü

results = []

for generation in range(num_generations):
    print(f"Generation {generation + 1}")
    fitness = []
    for i, network_params in enumerate(population):
        model = create_network(network_params)
        accuracy = compute_fitness(model, X_train, y_train, X_val, y_val)
        fitness.append(accuracy)
        print(f"Network {i + 1}: Accuracy {accuracy}  | Params {network_params} |  ")
        results.append({'generation': generation, 'network_params': network_params, 'accuracy': accuracy})
       
    parents = select_parents(population, fitness, num_parents)
    offspring_crossover = crossover(parents)
    offspring_mutation = mutate(offspring_crossover)

    population = parents + offspring_mutation

Generation 1
Network 1: Accuracy 0.9461666941642761  | Params {'first_layer_neurons': 128, 'hidden_layers': 3, 'hidden_layer_neurons': 128, 'activation': 'relu'} |  
Network 2: Accuracy 0.8711666464805603  | Params {'first_layer_neurons': 128, 'hidden_layers': 2, 'hidden_layer_neurons': 32, 'activation': 'tanh'} |  
Network 3: Accuracy 0.9223333597183228  | Params {'first_layer_neurons': 32, 'hidden_layers': 2, 'hidden_layer_neurons': 32, 'activation': 'relu'} |  
Network 4: Accuracy 0.9045000076293945  | Params {'first_layer_neurons': 128, 'hidden_layers': 1, 'hidden_layer_neurons': 32, 'activation': 'relu'} |  
Network 5: Accuracy 0.8575000166893005  | Params {'first_layer_neurons': 64, 'hidden_layers': 2, 'hidden_layer_neurons': 128, 'activation': 'tanh'} |  
Network 6: Accuracy 0.9271666407585144  | Params {'first_layer_neurons': 32, 'hidden_layers': 3, 'hidden_layer_neurons': 64, 'activation': 'relu'} |  
Network 7: Accuracy 0.8441666960716248  | Params {'first_layer_neurons': 64,

In [6]:
# datafreame çevirme

df = pd.DataFrame(results , columns = ['generation', 'network_params', 'accuracy'])

df

,generation,network_params,accuracy
0,0,"{'first_layer_neurons': 128, 'hidden_layers': ...",0.946167
1,0,"{'first_layer_neurons': 128, 'hidden_layers': ...",0.871167
2,0,"{'first_layer_neurons': 32, 'hidden_layers': 2...",0.922333
3,0,"{'first_layer_neurons': 128, 'hidden_layers': ...",0.904500
4,0,"{'first_layer_neurons': 64, 'hidden_layers': 2...",0.857500
5,0,"{'first_layer_neurons': 32, 'hidden_layers': 3...",0.927167
6,0,"{'first_layer_neurons': 64, 'hidden_layers': 1...",0.844167
7,0,"{'first_layer_neurons': 64, 'hidden_layers': 3...",0.824333
8,0,"{'first_layer_neurons': 32, 'hidden_layers': 2...",0.918833
9,0,"{'first_layer_neurons': 32, 'hidden_layers': 1...",0.888333
